In [1]:
import pandas as pd
import numpy as np
import json 
import re

In [2]:
!ls

Untitled.ipynb               load_data.pde
data                         placement_functions.pde
energy_production_assgn4.pde state_class.pde


In [93]:
df = pd.read_csv("../annual_generation_state.csv")
df.columns = ['_'.join([i.lower() for i in x.split()]) for x in df.columns]
df = df.iloc[:,:5]
df.head(20)

,year,state,type_of_producer,energy_source,generation_(megawatthours)
0,1990,AK,Total Electric Power Industry,Total,"5,599,506"
1,1990,AK,Total Electric Power Industry,Coal,"510,573"
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,"974,521"
3,1990,AK,Total Electric Power Industry,Natural Gas,"3,466,261"
4,1990,AK,Total Electric Power Industry,Petroleum,"497,116"
5,1990,AK,Total Electric Power Industry,Wind,0
6,1990,AK,Total Electric Power Industry,Wood and Wood Derived Fuels,"151,035"
7,1990,AK,"Electric Generators, Electric Utilities",Total,"4,493,024"
8,1990,AK,"Electric Generators, Electric Utilities",Coal,"311,960"
9,1990,AK,"Electric Generators, Electric Utilities",Hydroelectric Conventional,"974,521"


In [136]:
sample = pd.read_csv("data/1990_CA.csv", header=None).set_index(0)[1]
sample.head()

0
state                  CA
year                 1990
coal              8111529
geothermal       14521254
hydroelectric    24778819
Name: 1, dtype: object

In [95]:
red_df = \
    df[df['type_of_producer'] == 'Total Electric Power Industry']\
    .drop('type_of_producer', axis=1)

In [96]:
df['energy_source'].value_counts()

Total                             7762
Natural Gas                       6926
Petroleum                         6670
Coal                              5144
Other Biomass                     4558
Hydroelectric Conventional        4251
Wood and Wood Derived Fuels       3156
Other                             2802
Other Gases                       2000
Nuclear                           1866
Wind                              1840
Pumped Storage                    1118
Solar Thermal and Photovoltaic    1035
Geothermal                         391
Name: energy_source, dtype: int64

In [97]:
sample.columns

Index(['state', 'CA'], dtype='object')

In [143]:
all_states = []
index = df['energy_source'].unique()
for year in red_df['year'].unique():
    for state in red_df['state'].unique():
        if len(state.strip()) != 2:
            continue
        state_row = []
        row_df = red_df\
                    .query("year == {} & state == '{}'".format(year, state))\
                    .set_index('energy_source')['generation_(megawatthours)']\
                    .to_dict()
        for source in index:

            try:
                row_df[source] = int(re.sub("[^0-9.]", "", row_df[source]))
            except KeyError:
                row_df[source] = 0
        state_row.append(state)        
        state_row.append(year)
        state_row.append(
            row_df['Coal']\
            +   row_df['Petroleum']\
            +   row_df['Other']
        )
        state_row.append(
            row_df['Geothermal']
        )
        state_row.append(
            row_df['Hydroelectric Conventional']\
            +   row_df['Pumped Storage']
        )
        state_row.append(
            row_df['Natural Gas']\
            +   row_df['Other Gases']
        )
        state_row.append(
            row_df['Nuclear']
        )
        state_row.append(
            row_df['Wood and Wood Derived Fuels']\
            +   row_df['Other Biomass']
        )
        state_row.append(
            row_df['Solar Thermal and Photovoltaic']
        )
        state_row.append(
            row_df['Total']
        )
        state_row.append(
            row_df['Wind']
        )
        all_states.append(state_row)

In [144]:
print(all_states[30])

['NH', 1990, 5305696, 0, 1881087, 0, 4080691, 1137722, 0, 0, 12405196]


In [145]:
new_index = ['state'] + list(sample['state'])

In [146]:
new_index = sample.index

In [147]:
new_index

Index(['state', 'year', 'coal', 'geothermal', 'hydroelectric', 'natural_gas',
       'nuclear', 'biomass', 'solar', 'total', 'wind'],
      dtype='object', name=0)

In [148]:
for i in all_states:
    open('data/{}_{}.csv'.format(i[1], i[0]), 'w')\
        .write(','.join([str(x) for x in new_index]) 
               + '\n' 
               + ','.join([str(x) for x in i]))
#     pd.Series(data= i, index= new_index)\
#         .to_csv('data/{}_{}.csv'.format(i[1], i[0]))